### Make the necessary imports (libraries)

In [ ]:
# Import necessary libraries
import cnn_utils
from keras import layers, models
from keras.src.legacy.preprocessing.image import ImageDataGenerator


### Load the data

In [ ]:
data_dict = cnn_utils.load_cifar10_from_tar()

### Preprocess the data

In [ ]:
data = cnn_utils.preprocess_data(data_dict)

### Let's do a quick visualization of sample images (to also ensure we still have the correct shape)

In [ ]:
cnn_utils.visualize_data_samples(data)

### Augmentation of the data
Mminimal/conservative augmentation designed to be very safe:
- Tiny transformations: Only 5% shifts and 5° rotation (vs typical 10-15%)
- Basic flip: Horizontal flip only (doubles dataset safely)

In [ ]:
def apply_mild_data_augmentation():
    """Apply very conservative data augmentation"""
    return ImageDataGenerator(
        horizontal_flip=True,          # Only horizontal flip
        width_shift_range=0.05,        # Very small shifts (5%)
        height_shift_range=0.05,       # Very small shifts (5%)
        rotation_range=5               # Very small rotation (5 degrees)
    )

augmentation = apply_mild_data_augmentation()
augmentation.fit(data['X_train'])

### Let's define our CNN model (architect)
Modern VGG-style CNN with consistent regularization patterns:

- 3 blocks of paired convolutions (32→64→128 filters) following VGG's "double conv + pool" pattern
- Modern techniques: BatchNormalization and explicit Activation layers (instead of inline activation)
- GlobalAveragePooling instead of Flatten (reduces overfitting)
- Progressive dropout (0.25→0.25→0.5)
- Lightweight classifier (128 neurons vs typical 512+)


In [ ]:
# Define the CNN model architecture
def create_cnn_model(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential()

    # First block - keep successful pattern
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(32, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))  # Slightly lower dropout

    # Second block
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))

    # Third block
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))

    # Keep the successful GlobalAveragePooling
    model.add(layers.GlobalAveragePooling2D())

    # Simplified classifier
    model.add(layers.Dense(128))  # Even smaller
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))  # Lower dropout for stability
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [ ]:
model = create_cnn_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_utils.print_model_summary(model)


In [ ]:
from keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) #,
]

history = cnn_utils.train_model(model, data, augmentation=augmentation, callbacks=callbacks)


### Let's show the evaluation result

In [ ]:
cnn_utils.evaluate_model(model, data, history)